In [16]:
!pip install fasttext

## fasttext

facebook에서 개발한 라이브러리

word embeddings text Classification 모델을 학습할 수 있음

word embedding을 위해서 뉴럴 네트워크를 사용한다고 함

In [17]:
import fasttext

import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/MyDrive/소설작가분류/train.csv')
test = pd.read_csv('/content/drive/MyDrive/소설작가분류/test_x.csv')
submission = pd.read_csv('/content/drive/MyDrive/소설작가분류/sample_submission.csv')

In [18]:
file = open('fasttexttrain.txt', 'w+') #w+ 읽기 또는 쓰기 모드, 파일 없으면 새로 만듦
for i in train.index:
    line = '__label__' + str(train['author'][i]) + ' ' +train['text'][i]
    file.write(line+'\n')

In [19]:
#pre-trained된 분류기를 로드해서 모델 생성
text_clf_model = fasttext.train_supervised('fasttexttrain.txt', epoch=30, minCount=2, maxn=10, verbose=0)

result = text_clf_model.predict()

In [21]:
result = text_clf_model.predict('He was almost choking. There was so much, so much he wanted to say, but strange exclamations were all that came from his lips. The Pole gazed fixedly at him, at the bundle of notes in his hand; looked at odin, and was in evident perplexity.', k=5)
print(result)

# k==> 분류기로부터 상위 라벨 k개. 

# fasttext 분류기의 params.

# Text Classification
# input             # training file path (required)
# model             # unsupervised fasttext model {cbow, skipgram} [skipgram]
# lr                # learning rate [0.05]
# dim               # size of word vectors [100]
# ws                # size of the context window [5]
# epoch             # number of epochs [5]
# minCount          # minimal number of word occurences [5]
# minn              # min length of char ngram [3]
# maxn              # max length of char ngram [6]
# neg               # number of negatives sampled [5]
# wordNgrams        # max length of word ngram [1]
# loss              # loss function {ns, hs, softmax, ova} [ns]
# bucket            # number of buckets [2000000]
# thread            # number of threads [number of cpus]
# lrUpdateRate      # change the rate of updates for the learning rate [100]
# t                 # sampling threshold [0.0001]
# verbose           # verbose [2]

(('__label__3', '__label__0', '__label__2'), array([9.97198641e-01, 1.46512245e-03, 8.68424831e-04]))


In [23]:
for i in test.index:
    label, proba = text_clf_model.predict(test['text'][i], k=5)
    for la, pr in zip(label, proba):
        if '__label__0' == la:
            submission.loc[i,'0'] = pr
        elif '__label__1' == la:
            submission.loc[i, '1'] = pr
        elif '__label__2' == la:
            submission.loc[i, '2'] = pr
        elif '__label__3' == la:
            submission.loc[i, '3'] = pr
        elif '__label__4' == la:
            submission.loc[i, '4'] = pr

In [24]:
submission

,index,0,1,2,3,4
0,0,0.025707,0.441581,0.323085,0.206897,0.002780
1,1,0.112356,0.837495,0.001129,0.014095,0.034975
2,2,0.890749,0.055735,0.011897,0.021887,0.019781
3,3,0.023307,0.000837,0.925364,0.002232,0.048310
4,4,0.659009,0.102127,0.032219,0.177349,0.029346
...,...,...,...,...,...,...
19612,19612,0.022107,0.974342,0.000387,0.002294,0.000920
19613,19613,0.302090,0.009734,0.049494,0.013629,0.625102
19614,19614,0.017620,0.975687,0.000294,0.006073,0.000376
19615,19615,0.014061,0.923946,0.048835,0.010064,0.003144


In [26]:
submission.to_csv('result_fasttext.csv', index=False)
